In [0]:
%sql
CREATE TABLE delta.`/tmp/delta-table` USING DELTA AS SELECT col1 as id FROM VALUES 0,1,2,3,4;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM delta.`/tmp/delta-table`;

id
0
1
2
3
4


In [0]:
%sql
INSERT OVERWRITE delta.`/tmp/delta-table` SELECT col1 as id FROM VALUES 5,6,7,8,9;

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
-- Update every even value by adding 100 to it
UPDATE delta.`/tmp/delta-table` SET id = id + 100 WHERE id % 2 == 0;

num_affected_rows
2


In [0]:
%sql
-- Upsert (merge) new data
CREATE TEMP VIEW newData AS SELECT col1 AS id FROM VALUES 1,3,5,7,9,11,13,15,17,19;

In [0]:
%sql
MERGE INTO delta.`/tmp/delta-table` AS oldData
USING newData
ON oldData.id = newData.id
WHEN MATCHED
  THEN UPDATE SET id = newData.id
WHEN NOT MATCHED
  THEN INSERT (id) VALUES (newData.id);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,3,0,7


In [0]:
%sql
SELECT * FROM delta.`/tmp/delta-table`;

id
106
108
1
3
5
7
9
11
13
15


In [0]:
#Write a stream of data to a table
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table")

In [0]:
#Read a stream of changes from a table
stream2 = spark.readStream.format("delta").load("/tmp/delta-table").writeStream.format("console").start()